# SESSION_1

The objectives of this session is to handle jupyter notebook by instaling conda packages, know more abouit the fast5 Format, bascall  reads with Guppy and fimaly try to evaluate the accurary of Nanopore reads using Rattlesnake

Prerequisites: In a `terminal` (not here), You need to create the curso_1 env, to install biopython and jupyter and activate the `Conda` env as follow before to start jupyter

!conda create -y --name curso_1

!conda install -y -n curso_1 -c conda-forge biopython 

!conda install -y -n curso_1 -c anaconda h5py seaborn wget cmake

!conda activate curso_1

# Introduction about OXFORD NANOPORE

`Oxford Nanopore` Technologies is one of the leading companies in sequencing technologies providing direct DNA/RNA sequencing molecules in a portable way. A large variety of sequencer options and sample preparation protocols enabled the process of obtaining long genomic fragments (https://nanoporetech.com/products).  
The immense increase in fragment lengths, with respect to older sequencing generations, greatly facilitate the problem of de novo genome assembly by spanning even the longest repetitive regions. The limitation is the high error rate of the sequencing yield, ranging between 5 % and 15 %. This Notebook is intented to get familiar how the sequencing technology works, which format is used to store the sequencer output and how to translate it to genomic sequences.

The best way to see how Oxford Nanopore sequencing works is through their promotional videos. Run the cell below to load the YouTube video. (`command + enter` or `execute`)

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('RcP85JHLmnI', 560, 315)

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('iT_A_ucWMls', 560, 315)

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('ZE5L_ykAMj8', 560, 315) 

A ressource center with protocols and workflow is available at https://nanoporetech.com/resource-centre

Example:  
A workfow for Metagenomic assembly: https://nanoporetech.com/sites/default/files/s3/literature/metagenomic-assembly-workflow.pdf;  
A workflow for human genome assemby: https://nanoporetech.com/sites/default/files/s3/literature/human-genome-assembly-workflow.pdf;   
Genome assembly advices: https://nanoporetech.com/applications/investigation/assembly  

Additional information about the technology can be found at https://nanoporetech.com/community. The website features detailed guides for a plethora of sample preparation protocols and hosts the needed software for both sequencing and basecalling.  
Please visit the tutorial page at https://community.nanoporetech.com/knowledge/bioinformatics for more information

# TO KNOW THE fast5 FORMAT

To get a better grasp of the whole matter, we prepared a small dataset of a Bacillus subtilis genome which was sequenced with the R10 nanopore. The output of any Oxford Nanopore sequencer is stored in `fast5` format -or multi `fast5` because it contains a batch of reads in a single file. 
See : [FAST5](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwjl78ag6YL5AhWtQTABHWM8Cu8QFnoECAIQAw&url=https%3A%2F%2Fwww.biorxiv.org%2Fcontent%2Fbiorxiv%2Fearly%2F2021%2F06%2F30%2F2021.06.29.450255%2FDC2%2Fembed%2Fmedia-2.pdf%3Fdownload%3Dtrue&usg=AOvVaw1Qc6P-3YAJRPhjk-PVh89o) file.  `fast5` format  is a specialization of the [hdf5](https://www.hdfgroup.org/solutions/hdf5/) file format.  
It is a hierarchical data format, which looks like a filesystem, used to store and organize large datasets. The format is supported in a wide range of programing languages, and the Python API is incorporated in the `h5py` module.

In [ ]:
import h5py

fast5 = h5py.File('data/bacillus_subtilis/bs_raw.fast5', mode='r')
fast5.name

Hierarchy of a Multi-fast5 file:  
The root group (e.g. /) contains a group for each read that is named as “read_” followed by the read identifier

In [ ]:
list(fast5.keys())

The object we loaded from the fast5 file is treated as a simple Python dictionary. To get a read, we simply put the desired read name inside brackets `[]`. We can also see the whole object tree with the visit member function. In addition, each object has meta data in its `attrs` member variable.

In [ ]:
Under each read group, there are different groups:  

In [ ]:
read = fast5['read_003f003d-a943-48b6-bf18-31f89f9fbed6']
read.visit(print)

Each group has different Attribute name and variable:  

In [ ]:
print(read['tracking_id'].attrs['flow_cell_id'])
print(read['tracking_id'].attrs['device_type'])
print(read['tracking_id'].attrs['protocol_group_id'])
print(read['tracking_id'].attrs['protocol_run_id'])
print(read['tracking_id'].attrs['protocols_version'])

The point of interest is the raw electric signal which will be latter translated with the Guppy basecaller into a DNA sequence.

In [ ]:
signal = read['Raw']['Signal']
signal.shape

In [ ]:
signal[:]

In [ ]:
from matplotlib import pyplot
import seaborn

seaborn.set()
seaborn.set_style("white")

pyplot.rcParams["figure.figsize"] = (20,5)

pyplot.plot(signal[:])

In [ ]:
pyplot.plot(signal[0:5000])

In [ ]:
pyplot.plot(signal[4000:5000])

We can easily create a smaller subset of the set we have at the moment. We need to open a new hdf5 file and copy the desired reads to it as below.

In [ ]:
read_f = h5py.File('read.fast5', mode='w')
fast5.copy(read, read_f)
read_f.close()

You wrote the desire read as read.fadt5

In [ ]:
!ls -lrt


In order to basecall the data we need to download the Guppy basecaller with the following command (pick the appropriate commands with regards to your operating system) and unpack the archive. To see if it is working properly run it without parameters.

# TO BASECALL READS WITH `GUPPY`

[How works basecalling](https://nanoporetech.com/how-it-works/basecalling)

[Guppy](https://github.com/asadprodhan/GPU-accelerated-guppy-basecalling)

[Tutorial](https://timkahlke.github.io/LongRead_tutorials/BS_G.html)

In [ ]:
#for Linux users:  
!wget https://mirror.oxfordnanoportal.com/software/analysis/ont-guppy-cpu_4.4.1_linux64.tar.gz
#!tar -xzvf ont-guppy-cpu_4.4.1_linux64.tar.gz

#**#For MAC users only**
#!wget https://mirror.oxfordnanoportal.com/software/analysis/ont-guppy-cpu_4.4.1_osx64.zip
#!unzip ont-guppy-cpu_4.4.1_osx64.zip

In [ ]:
!ls

unzip the downloaded file if not done automaticaly by the system

Call the `guppy basecaller` program to see usage

In [ ]:
!ont-guppy-cpu/bin/guppy_basecaller

In [ ]:
!ls ont-guppy-cpu/data

The basecalling parameters depend on which flowcell and library preparation kit was used to produce the data

In [ ]:
!ont-guppy-cpu/bin/guppy_basecaller --print_workflows

First we will try to translate only one signal in the subset file we created. The below Guppy command searches the current directory for all fast5 files and translates them to [FASTQ](https://en.wikipedia.org/wiki/FASTQ_format) format, using the R10 configuration file.

In [ ]:
!cat ont-guppy-cpu/data/dna_r10_450bps_fast.cfg

In [ ]:
!ls

In [ ]:
!ont-guppy-cpu/bin/guppy_basecaller \
    -c ont-guppy-cpu/data/dna_r10_450bps_fast.cfg \
    -i data/bacillus_subtilis \
    -s guppy_read

In [ ]:
!cat guppy_read/fastq_runid_*.fastq

In [ ]:
!cat guppy_read/sequencing_summary.txt

In [ ]:
!ls guppy_all

We can obtain the basecalled read length with the command below.

In [ ]:
!head -n 2 guppy_read/fastq_runid_*.fastq | tail -n 1 | wc

Usually, the basecalling is accompanied with quality control, classifying reads into pass/faill classes.  
Filtering is done with regards to the reads average Phred quality score. For each base, the score is calculated with: $Q = -10log_{10}{P}$, where $P$ is the base-calling error probability.  
For example, a Phred score of $50$ equals the base call accuracy of $99.999\%$.  See [Phred_quality_score](https://en.wikipedia.org/wiki/Phred_quality_score). 
Guppy has options `--qscore_filtering` and `--min_qscore`, first enabling filtering and the second setting the threshold (which is 7 by default). To store all found data into a single fastq file use option `-q 0`. Let us now try to basecall the whole dataset.

In [ ]:
!ont-guppy-cpu/bin/guppy_basecaller \
    -c ont-guppy-cpu/data/dna_r10_450bps_fast.cfg \
    -i data/bacillus_subtilis/ \
    -s guppy_all \
    -q 0 \
    --qscore_filtering \
    --min_qscore 7

In [ ]:
!ls guppy_all

In [ ]:
!cat guppy_all/sequencing_summary.txt

To see the the length distribution of the basecalled read set we can simply draw a histogram as below. In addition, it would be nice to calculate the average base accuracy given a reference genome. Lets do that for one read that passed the filtering.

Exercise
If the command below doesn't work it means that Biopython is not install on you system. Biopython needs to be installed  (https://biopython.org/wiki/Download). In this case use conda in the window below to install Biopython

In [ ]:
from Bio import SeqIO

with open('guppy_all/pass/fastq_runid_aea170ff77e7e76bd42c58335c528e15f49ba94c_0_0.fastq') as handle:
    reads = []
    for read in SeqIO.parse(handle, 'fastq'):
        reads.append(len(read.seq))
    pyplot.hist(reads)

Lets pick read `04f26824-995e-49df-b661-7bf4efcc8ce9` for example.  
First we need to find where in the reference did it originate. This is done with tools called mappers which will be explained in the `session_2`.  
For now we can assume we know the positions are $[2158952, 2182735]$. We need to extract the read and the portion of the reference into new files. Afterwards we will align them with edit distance to calculate the accuracy.

# TO KNOW ACCURACY WITH RATLESNAKE, using a reference genome

`Ratlesnake` was designed to aid development of new assembly algorithms. Given a reference genome, it calculates the most contiguous assembly possible for each chromosome separately. In addition, it classifies sequences into distinct classes and annotates related events, such as breaking points in chimeric sequences, inclusion intervals of contained sequences and repetitive genomic regions in sequences overlapping them. [Ratlesnake](https://github.com/lbcb-sci/ratlesnake)

You need to install cmake before on your system if not done ! 

In [ ]:
!git clone https://github.com/lbcb-sci/ratlesnake
!mkdir ratlesnake/build
!cmake -S ratlesnake -B ratlesnake/build -DCMAKE_BUILD_TYPE=Release
!make -C ratlesnake/build

In [ ]:
!ratlesnake/build/bin/ratlesnake

In [ ]:
!ratlesnake/build/bin/ratlesnake \
    guppy_all/pass/fastq_runid_aea170ff77e7e76bd42c58335c528e15f49ba94c_0_0.fastq

In [ ]:
!ratlesnake/build/bin/ratlesnake \
    -r 1 \
    -t 4 \
    guppy_all/pass/fastq_runid_aea170ff77e7e76bd42c58335c528e15f49ba94c_0_0.fastq \
    data/bacillus_subtilis/bs_ref.fasta
#-r 1 - sequence accuracy given reference (histogram on stdout)

We will use Guppy's high accuracy configuration file to basecall the first Bacillus subtilis reads set and compare the results with the fast mode.

Let's try `Guppy` with different conf files: dna_r10_450bps_fast.cfg and dna_r10_450bps_hac.cfg  
`Guppy` provides two different architectures: a fast base caller, which can base call with 85–92% median read accuracy and a high-accuracy base caller (90–96% median read accuracy).  

In [ ]:
!ont-guppy-cpu/bin/guppy_basecaller \
    -c ont-guppy-cpu/data/dna_r10_450bps_fast.cfg \
    -i data/bacillus_subtilis/ \
    -s guppy_fast \
    -q 0 \
    --qscore_filtering

In [ ]:
!ratlesnake/build/bin/ratlesnake \
    -r 0 -r 1 \
    -t 4 \
    guppy_fast/pass/fastq_*.fastq \
    data/bacillus_subtilis/bs_ref.fasta

In [ ]:
The folowing basecalling will take time! 

In [ ]:
!ont-guppy-cpu/bin/guppy_basecaller \
    -c ont-guppy-cpu/data/dna_r10_450bps_hac.cfg \
    -i data/bacillus_subtilis \
    -s guppy_hac \
    -q 0 \
    --qscore_filtering

In [ ]:
!ratlesnake/build/bin/ratlesnake \
    -r 0 -r 1 \
    -t 4 \
    guppy_hac/pass/fastq_*.fastq \
    data/bacillus_subtilis/bs_ref.fasta

Compare the results with different `Guppy` conf files